In [1]:
# 1. Install Java
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# 2. Download Spark 3.5.0 with Hadoop 3
!wget -q https://archive.apache.org/dist/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar xf spark-3.5.0-bin-hadoop3.tgz

# 3. Set environment variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

In [2]:
# 4. Install findspark
!pip install -q findspark
import findspark
findspark.init()

In [3]:
# 5. Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").appName("Colab-PySpark").getOrCreate()

In [4]:
spark

In [15]:
#create a dataframe
df = spark.read.csv("/content/sample_data/sample_census_data.csv",header=True,inferSchema=True)

In [16]:
df.show()

+----------+----------+----------+----------+--------------+----+
|     State|Population|Median_Age|Households|Average_Income|Year|
+----------+----------+----------+----------+--------------+----+
|California|  39538223|      36.5|  13000000|         78000|2020|
|     Texas|  29145505|      34.8|   9800000|         67000|2020|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|
|  New York|  20201249|      39.0|   7500000|         75000|2020|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|
+----------+----------+----------+----------+--------------+----+



In [17]:
df.printSchema()

root
 |-- State: string (nullable = true)
 |-- Population: integer (nullable = true)
 |-- Median_Age: double (nullable = true)
 |-- Households: integer (nullable = true)
 |-- Average_Income: integer (nullable = true)
 |-- Year: integer (nullable = true)



In [21]:
#  groupBy allows the use of SQL-like aggregations
result_df = df.groupBy("Year").agg({'Population':'sum'}).show()

+----+---------------+
|Year|sum(Population)|
+----+---------------+
|2020|      123235672|
+----+---------------+



In [25]:
result_df2 = df.groupBy("Year").agg({'Median_Age':'avg'}).show()

+----+------------------+
|Year|   avg(Median_Age)|
+----+------------------+
|2020|38.160000000000004|
+----+------------------+



In [29]:
#using filter and select
filtered_df= df.filter(df['State']=='Texas').select("Median_Age","Average_Income")

In [30]:
filtered_df.show()

+----------+--------------+
|Median_Age|Average_Income|
+----------+--------------+
|      34.8|         67000|
+----------+--------------+



In [32]:
#sort
df.sort("Average_Income").show()

+----------+----------+----------+----------+--------------+----+
|     State|Population|Median_Age|Households|Average_Income|Year|
+----------+----------+----------+----------+--------------+----+
|   Florida|  21538187|      42.0|   8000000|         61000|2020|
|     Texas|  29145505|      34.8|   9800000|         67000|2020|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|
|  New York|  20201249|      39.0|   7500000|         75000|2020|
|California|  39538223|      36.5|  13000000|         78000|2020|
+----------+----------+----------+----------+--------------+----+



In [35]:
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType

In [36]:
schema = StructType([
    StructField("State", StringType(), True),
    StructField("Population", IntegerType(), True),
    StructField("Median_Age", DoubleType(), True),
    StructField("Households", IntegerType(), True),
    StructField("Average_Income", IntegerType(), True),
    StructField("Year", IntegerType(), True)
])

In [37]:
# Create a new row ( Median_Age = NULL)
new_row_data = [("New Jersey", 3104614, None, 1200000, 65000, 2020)]

In [60]:
# Create a new DataFrame with the new row
new_row_df = spark.createDataFrame(new_row_data, schema)

# Add the new row to the existing DataFrame
updated_df = df.union(new_row_df)

# Show the updated DataFrame
updated_df.show()

+----------+----------+----------+----------+--------------+----+
|     State|Population|Median_Age|Households|Average_Income|Year|
+----------+----------+----------+----------+--------------+----+
|California|  39538223|      36.5|  13000000|         78000|2020|
|     Texas|  29145505|      34.8|   9800000|         67000|2020|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|
|  New York|  20201249|      39.0|   7500000|         75000|2020|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|
|New Jersey|   3104614|      NULL|   1200000|         65000|2020|
+----------+----------+----------+----------+--------------+----+



In [40]:
#Data manipulation with DataFrames
updated_df=updated_df.na.fill({"Median_Age":38.0})

#use .na.drop() ->    to remove rows with null values
#df.where(col(" column_name").isNotNull())->   filter out null values

In [41]:
updated_df.show()

+----------+----------+----------+----------+--------------+----+
|     State|Population|Median_Age|Households|Average_Income|Year|
+----------+----------+----------+----------+--------------+----+
|California|  39538223|      36.5|  13000000|         78000|2020|
|     Texas|  29145505|      34.8|   9800000|         67000|2020|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|
|  New York|  20201249|      39.0|   7500000|         75000|2020|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|
|New Jersey|   3104614|      38.0|   1200000|         65000|2020|
+----------+----------+----------+----------+--------------+----+



**Column Operations**

In [44]:
#create a new column
updated_df=updated_df.withColumn("Other_Income",updated_df["Average_Income"]-40000)

In [45]:
updated_df.show()

+----------+----------+----------+----------+--------------+----+------------+
|     State|Population|Median_Age|Households|Average_Income|Year|Other_Income|
+----------+----------+----------+----------+--------------+----+------------+
|California|  39538223|      36.5|  13000000|         78000|2020|       38000|
|     Texas|  29145505|      34.8|   9800000|         67000|2020|       27000|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|       21000|
|  New York|  20201249|      39.0|   7500000|         75000|2020|       35000|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|       28000|
|New Jersey|   3104614|      38.0|   1200000|         65000|2020|       25000|
+----------+----------+----------+----------+--------------+----+------------+



In [47]:
#Rename columns
updated_df=updated_df.withColumnRenamed("Other_Income","Extra_Income")

In [48]:
updated_df.show()

+----------+----------+----------+----------+--------------+----+------------+
|     State|Population|Median_Age|Households|Average_Income|Year|Extra_Income|
+----------+----------+----------+----------+--------------+----+------------+
|California|  39538223|      36.5|  13000000|         78000|2020|       38000|
|     Texas|  29145505|      34.8|   9800000|         67000|2020|       27000|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|       21000|
|  New York|  20201249|      39.0|   7500000|         75000|2020|       35000|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|       28000|
|New Jersey|   3104614|      38.0|   1200000|         65000|2020|       25000|
+----------+----------+----------+----------+--------------+----+------------+



In [50]:
#Remove unnecessary column
updated_df=updated_df.drop("Extra_Income").show()

+----------+----------+----------+----------+--------------+----+
|     State|Population|Median_Age|Households|Average_Income|Year|
+----------+----------+----------+----------+--------------+----+
|California|  39538223|      36.5|  13000000|         78000|2020|
|     Texas|  29145505|      34.8|   9800000|         67000|2020|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|
|  New York|  20201249|      39.0|   7500000|         75000|2020|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|
|New Jersey|   3104614|      38.0|   1200000|         65000|2020|
+----------+----------+----------+----------+--------------+----+



**Row Operations**

In [61]:
filtered_df = updated_df.filter(updated_df["Average_Income"] > 70000)
filtered_df.show()


+----------+----------+----------+----------+--------------+----+
|     State|Population|Median_Age|Households|Average_Income|Year|
+----------+----------+----------+----------+--------------+----+
|California|  39538223|      36.5|  13000000|         78000|2020|
|  New York|  20201249|      39.0|   7500000|         75000|2020|
+----------+----------+----------+----------+--------------+----+



In [ ]:
#Also you can use groupBy and Aggregate functions

#JOINS IN PYSPARK

In [62]:
# New dataset
data2 = [
    ("California", "West", 5.2),
    ("Texas", "South", 4.1),
    ("Florida", "South", 3.8),
    ("New York", "Northeast", 5.9),
    ("Illinois", "Midwest", 4.7),
    ("Nevada", "West", 5.5),
    ("Ohio", "Midwest", 4.4)
]

columns2 = ["State", "Region", "Unemployment_Rate"]

df2 = spark.createDataFrame(data2, columns2)
df2.show()

+----------+---------+-----------------+
|     State|   Region|Unemployment_Rate|
+----------+---------+-----------------+
|California|     West|              5.2|
|     Texas|    South|              4.1|
|   Florida|    South|              3.8|
|  New York|Northeast|              5.9|
|  Illinois|  Midwest|              4.7|
|    Nevada|     West|              5.5|
|      Ohio|  Midwest|              4.4|
+----------+---------+-----------------+



In [63]:
#inner join -->Only rows where State exists in both DataFrames
inner_join = updated_df.join(df2, on="State", how="inner")
inner_join.show()


+----------+----------+----------+----------+--------------+----+---------+-----------------+
|     State|Population|Median_Age|Households|Average_Income|Year|   Region|Unemployment_Rate|
+----------+----------+----------+----------+--------------+----+---------+-----------------+
|California|  39538223|      36.5|  13000000|         78000|2020|     West|              5.2|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|    South|              3.8|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|  Midwest|              4.7|
|  New York|  20201249|      39.0|   7500000|         75000|2020|Northeast|              5.9|
|     Texas|  29145505|      34.8|   9800000|         67000|2020|    South|              4.1|
+----------+----------+----------+----------+--------------+----+---------+-----------------+



In [64]:
#left join-->All rows from updated_df, plus matches from df2
left_join = updated_df.join(df2, on="State", how="left")
left_join.show()


+----------+----------+----------+----------+--------------+----+---------+-----------------+
|     State|Population|Median_Age|Households|Average_Income|Year|   Region|Unemployment_Rate|
+----------+----------+----------+----------+--------------+----+---------+-----------------+
|     Texas|  29145505|      34.8|   9800000|         67000|2020|    South|              4.1|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|  Midwest|              4.7|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|    South|              3.8|
|California|  39538223|      36.5|  13000000|         78000|2020|     West|              5.2|
|  New York|  20201249|      39.0|   7500000|         75000|2020|Northeast|              5.9|
|New Jersey|   3104614|      NULL|   1200000|         65000|2020|     NULL|             NULL|
+----------+----------+----------+----------+--------------+----+---------+-----------------+



In [65]:
#right join -->All rows from df2, plus matches from updated_df
right_join = updated_df.join(df2, on="State", how="right")
right_join.show()


+----------+----------+----------+----------+--------------+----+---------+-----------------+
|     State|Population|Median_Age|Households|Average_Income|Year|   Region|Unemployment_Rate|
+----------+----------+----------+----------+--------------+----+---------+-----------------+
|     Texas|  29145505|      34.8|   9800000|         67000|2020|    South|              4.1|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|    South|              3.8|
|California|  39538223|      36.5|  13000000|         78000|2020|     West|              5.2|
|      Ohio|      NULL|      NULL|      NULL|          NULL|NULL|  Midwest|              4.4|
|    Nevada|      NULL|      NULL|      NULL|          NULL|NULL|     West|              5.5|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|  Midwest|              4.7|
|  New York|  20201249|      39.0|   7500000|         75000|2020|Northeast|              5.9|
+----------+----------+----------+----------+--------------+

In [66]:
#full_join --> All rows from both DataFrames, with NULLs where no match
full_join = updated_df.join(df2, on="State", how="outer")
full_join.show()


+----------+----------+----------+----------+--------------+----+---------+-----------------+
|     State|Population|Median_Age|Households|Average_Income|Year|   Region|Unemployment_Rate|
+----------+----------+----------+----------+--------------+----+---------+-----------------+
|California|  39538223|      36.5|  13000000|         78000|2020|     West|              5.2|
|   Florida|  21538187|      42.0|   8000000|         61000|2020|    South|              3.8|
|  Illinois|  12812508|      38.5|   4900000|         68000|2020|  Midwest|              4.7|
|    Nevada|      NULL|      NULL|      NULL|          NULL|NULL|     West|              5.5|
|New Jersey|   3104614|      NULL|   1200000|         65000|2020|     NULL|             NULL|
|  New York|  20201249|      39.0|   7500000|         75000|2020|Northeast|              5.9|
|      Ohio|      NULL|      NULL|      NULL|          NULL|NULL|  Midwest|              4.4|
|     Texas|  29145505|      34.8|   9800000|         67000|

UNION

*   Both DataFrames have exactly the same number of columns, and
*  The columns are in the same order and data types.










In [ ]:
#eg-> df_union=df1.union(df2)